In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

from glob import glob

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 512
r_h = 512

total_pix = r_w * r_h * channel

In [4]:
data_txt = open('./data_split/data_dir_dict_0614.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [5]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data[i,:] = img2.flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [6]:
ev_dir2 = [y for x in os.walk('/data_oh_rgb/test-images/') for y in glob(os.path.join(x[0], '*.png'))]

In [7]:
ev_data2 = np.zeros((len(ev_dir2), total_pix), dtype=np.float32)
ev_label2 = np.zeros((len(ev_dir2), 1), dtype=np.int32)

In [8]:
for i in range(len(ev_dir2)):
    img = cv2.imread(ev_dir2[i])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data2[i,:] = img2.flatten()
    if 'normal' in ev_dir2[i]:
        ev_label2[i] = 0
    else: ev_label2[i] = 1

In [11]:
f_dir1 = '/models/tr_together/Train_2/model.ckpt-5192.meta'
f_dir2 = '/models/tr_together/Train_2/'

#f_dir1 = '/models/ICROS_re/Train_3/model.ckpt-2280.meta'
#f_dir2 = '/models/ICROS_re/Train_3/'

ev_in = ev_data
ev_in_label = ev_label

saver = tf.train.import_meta_graph(f_dir1)
batch = 32
ev_steps = int(ev_in_label.shape[0]/batch)
ev_acc = np.zeros(ev_in_label.shape[0])

with tf.Session() as sess:
    
    new_saver = tf.train.import_meta_graph(f_dir1)
    new_saver.restore(sess, tf.train.latest_checkpoint(f_dir2))
    graph = tf.get_default_graph()
    #for op in tf.get_default_graph().get_operations():
        #print(op.name)
        #print(op.values())
        
    data_in = graph.get_tensor_by_name("fifo_queue_DequeueUpTo:1")

    #feed_dict={data_in:ev_data[0:10], label:ev_label[0:10]}
    keep_prob = graph.get_tensor_by_name("dropout/dropout/keep_prob:0")
    for i in range(ev_steps):
        if i < (ev_steps-1):
            feed_dict={data_in:ev_in[batch*i:batch*(i+1)], keep_prob:1.0}
            label = ev_in_label[batch*i:batch*(i+1)].reshape(batch)
        else:
            feed_dict={data_in:ev_in[batch*i:], keep_prob:1.0}
            temp = ev_in_label[batch*i:]
            label = temp.reshape(len(temp))
        
        prob = graph.get_tensor_by_name("softmax_tensor:0")
        classes = graph.get_tensor_by_name("ArgMax:0")
        
        
        result_pr = sess.run(prob, feed_dict=feed_dict)
        result_cl = sess.run(classes, feed_dict=feed_dict)
        
        print(result_pr)
        print(result_cl)
        print(label)

        acc = np.equal(result_cl, label).reshape((1,len(label)))*1
        
        print(acc)
        print(np.mean(acc))
        
        if i < (ev_steps-1):
            ev_acc[batch*i:batch*(i+1)] = acc
        else:
            ev_acc[batch*i:] = acc


INFO:tensorflow:Restoring parameters from /models/tr_together/Train_2/model.ckpt-5192
[[4.6824658e-07 9.9999952e-01]
 [2.0840323e-04 9.9979156e-01]
 [1.0000000e+00 9.6840955e-15]
 [1.0000000e+00 7.0712485e-12]
 [1.0000000e+00 3.4506292e-21]
 [2.8904738e-05 9.9997115e-01]
 [3.2702604e-07 9.9999964e-01]
 [1.2678768e-04 9.9987316e-01]
 [1.0000000e+00 1.0483744e-12]
 [7.2674346e-05 9.9992728e-01]
 [2.2116803e-04 9.9977881e-01]
 [9.9982423e-01 1.7581078e-04]
 [1.0822892e-08 1.0000000e+00]
 [4.7389949e-03 9.9526101e-01]
 [6.7777096e-11 1.0000000e+00]
 [1.0000000e+00 4.9555244e-13]
 [1.3911425e-12 1.0000000e+00]
 [1.6034224e-13 1.0000000e+00]
 [1.0499614e-04 9.9989498e-01]
 [4.1129122e-05 9.9995887e-01]
 [9.9922407e-01 7.7596464e-04]
 [6.0733065e-08 9.9999988e-01]
 [9.5055187e-01 4.9448077e-02]
 [9.9998140e-01 1.8575574e-05]
 [9.2039041e-02 9.0796101e-01]
 [1.0000000e+00 5.2510494e-08]
 [6.4457080e-23 1.0000000e+00]
 [1.6068289e-04 9.9983931e-01]
 [9.9999964e-01 3.9031349e-07]
 [1.7681156e-04

[[3.3494805e-05 9.9996650e-01]
 [1.0000000e+00 5.0126213e-27]
 [1.0000000e+00 4.2018242e-14]
 [6.4444139e-09 1.0000000e+00]
 [9.9993825e-01 6.1688705e-05]
 [1.0000000e+00 2.2119688e-10]
 [1.2116700e-01 8.7883306e-01]
 [1.0000000e+00 1.6702771e-14]
 [1.9041826e-18 1.0000000e+00]
 [8.3711677e-07 9.9999917e-01]
 [1.0000000e+00 4.3325989e-11]
 [9.9999988e-01 8.7060677e-08]
 [1.0196309e-02 9.8980367e-01]
 [9.9998021e-01 1.9842864e-05]
 [2.1151762e-22 1.0000000e+00]
 [9.4686902e-01 5.3131025e-02]
 [1.0000000e+00 2.2733736e-10]
 [2.7990876e-29 1.0000000e+00]
 [9.9998844e-01 1.1525045e-05]
 [9.9998224e-01 1.7718296e-05]
 [6.6235256e-03 9.9337643e-01]
 [1.0000000e+00 4.5195783e-08]
 [1.0000000e+00 4.6732680e-15]
 [6.2643609e-05 9.9993730e-01]
 [1.0000000e+00 1.3759202e-14]
 [4.9549703e-07 9.9999952e-01]
 [1.0000000e+00 4.6527917e-16]
 [1.0000000e+00 7.9186682e-27]
 [3.7122682e-05 9.9996293e-01]
 [9.9999976e-01 2.8478658e-07]
 [9.9998939e-01 1.0565110e-05]
 [2.2051396e-05 9.9997795e-01]]
[1 0 0 

[[1.0000000e+00 4.4375504e-08]
 [5.8626019e-06 9.9999416e-01]
 [1.0000000e+00 4.9294243e-08]
 [2.4477215e-05 9.9997556e-01]
 [1.0000000e+00 2.0987973e-13]
 [1.0000000e+00 1.6584430e-21]
 [1.6459721e-11 1.0000000e+00]
 [6.1564258e-04 9.9938428e-01]
 [9.9251866e-01 7.4812779e-03]
 [9.9999690e-01 3.1190214e-06]
 [5.8263780e-08 1.0000000e+00]
 [9.9397981e-01 6.0202279e-03]
 [2.4596777e-06 9.9999750e-01]
 [1.0000000e+00 6.1768497e-15]
 [2.1867724e-02 9.7813225e-01]
 [1.0000000e+00 1.8535653e-08]
 [3.2130622e-12 1.0000000e+00]
 [4.0101275e-01 5.9898722e-01]
 [1.0000000e+00 4.6600508e-11]
 [5.4911961e-06 9.9999452e-01]
 [7.0790526e-11 1.0000000e+00]
 [1.0000000e+00 2.6771156e-25]
 [1.4064769e-03 9.9859351e-01]
 [1.0000000e+00 1.5601522e-09]
 [1.0000000e+00 2.2593419e-10]
 [4.2242099e-11 1.0000000e+00]
 [8.0123312e-12 1.0000000e+00]
 [4.8817805e-07 9.9999952e-01]
 [1.0000000e+00 7.4147296e-13]
 [9.9999893e-01 1.1028799e-06]
 [6.2921707e-04 9.9937075e-01]
 [2.0888229e-03 9.9791116e-01]]
[0 1 0 

[[3.48964386e-04 9.99651074e-01]
 [9.99901772e-01 9.82366400e-05]
 [1.64082041e-06 9.99998331e-01]
 [9.99868035e-01 1.31982582e-04]
 [1.00000000e+00 1.93223194e-12]
 [8.62211309e-05 9.99913812e-01]
 [9.99895573e-01 1.04380808e-04]
 [2.24458785e-10 1.00000000e+00]
 [1.00000000e+00 1.83765866e-18]
 [9.99999642e-01 3.84780748e-07]
 [9.99973774e-01 2.61765817e-05]
 [1.00000000e+00 4.05305520e-12]
 [1.00000000e+00 6.18961398e-14]
 [1.00000000e+00 3.98110669e-14]
 [9.99998808e-01 1.19000265e-06]
 [5.61674649e-04 9.99438345e-01]
 [7.79266050e-03 9.92207348e-01]
 [5.84283261e-17 1.00000000e+00]
 [5.38263339e-05 9.99946117e-01]
 [1.00000000e+00 1.04048619e-30]
 [9.98324335e-01 1.67561928e-03]
 [1.00000000e+00 6.59055528e-33]
 [2.15575453e-02 9.78442490e-01]
 [8.46337527e-03 9.91536617e-01]
 [1.00000000e+00 8.67064463e-14]
 [9.99882936e-01 1.17100441e-04]
 [1.94602256e-21 1.00000000e+00]
 [9.99999881e-01 8.99834021e-08]
 [1.00000000e+00 3.67394266e-12]
 [1.00000000e+00 3.45246581e-10]
 [9.999960

In [12]:
print(ev_acc[-1])
print(np.mean(ev_acc))
fails = np.where(ev_acc == 0)
print(len(fails[0]))

1.0
0.9814004376367614
17
